# Imports

In [7]:
%pip install accelerate

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

  Using cached accelerate-1.6.0-py3-none-any.whl.metadata (19 kB)
Using cached accelerate-1.6.0-py3-none-any.whl (354 kB)


In [1]:
from tqdm import tqdm
from langchain.schema import Document
import retrieval_utils
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
# from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA, LLMChain

import importlib
importlib.reload(retrieval_utils)

<module 'retrieval_utils' from 'c:\\Users\\ankum\\OneDrive\\Desktop\\vivado_rag\\retrieval_utils.py'>

# Setup

In [2]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization = True)
print("Total chunks in FAISS index:", vectorstore.index.ntotal)

C:\Users\ankum\AppData\Local\Temp\ipykernel_25480\3598431607.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


Total chunks in FAISS index: 1119


In [3]:
question_page_pairs = retrieval_utils.load_questions_from_csv("vivado_rag_question_bank.csv")
print(f"Loaded {len(question_page_pairs)} questions")

Loaded 10 questions


In [4]:
[qpp[0] for qpp in question_page_pairs]

['How can I add enumerations using TCL commands?',
 'What key settings can I use when generating an encrypted bitstream?',
 'I need to both watch a signal’s value in the Waveform window and use that same signal to kick off a capture. What probe type should I pick?',
 'How can I create a custom debug probe on an ILA core in the Vivado Hardware Manager?',
 'I need to both validate and optimize my FPGA’s high-speed serial links in-system and, at the same time, issue AXI reads and writes over JTAG without a processor.',
 'Can I get some information on how to do serial I/O hardware debugging in Vivado?',
 'I’d like to script an IBERT eye-scan on my KC705 board using Vivado’s Tcl interface',
 'When I enable Auto Re-trigger on an ILA core, I notice I sometimes miss signal cycles between captures. How can I fix that?',
 'How do I add probes from my ILA core into the Waveform window?',
 'How can I read on-chip temperature data at run time?']

In [5]:
# one-time download of the model
pipe = pipeline(
    "text-generation",
    model="bigscience/bloom-560m",
    device_map="auto",
    torch_dtype="auto",
    max_new_tokens=256,
)

local_llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cpu
C:\Users\ankum\AppData\Local\Temp\ipykernel_25480\705101214.py:10: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  local_llm = HuggingFacePipeline(pipeline=pipe)


In [6]:
template = """You are a Vivado expert. Use the following excerpts (each labeled with its page) to answer the question from a new user below.
When you quote a fact, provide the page number as well.

Excerpts:
{context}

Question: {question}
Answer:"""

prompt = PromptTemplate(
    input_variables = ["context", "question"],
    template = template
)

In [7]:
retriever_k5 = vectorstore.as_retriever(search_kwargs={"k": 5})
doc_chain = LLMChain(llm=local_llm, prompt=prompt)
qa_chain_k5 = RetrievalQA(
    retriever = retriever_k5,
    combine_documents_chain = doc_chain,
    return_source_documents = True,
    input_key = "query",
)

C:\Users\ankum\AppData\Local\Temp\ipykernel_15016\359740405.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  doc_chain = LLMChain(llm=local_llm, prompt=prompt)
C:\Users\ankum\AppData\Local\Temp\ipykernel_15016\359740405.py:3: LangChainDeprecationWarning: This class is deprecated. Use the `create_retrieval_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/retrieval_qa/
  qa_chain_k5 = RetrievalQA(


AttributeError: 'LLMChain' object has no attribute 'get'

In [7]:
# k = 5 gave 85% recall
retriever_k5 = vectorstore.as_retriever(search_kwargs={"k": 5})
qa_chain_k5 = RetrievalQA.from_chain_type(
    llm = local_llm,
    chain_type = "stuff" ,
    retriever = retriever_k5,
    return_source_documents = True,
    chain_type_kwargs = {"prompt": prompt}
)

In [8]:
# k = 12 gave 90% recall
retriever_k12 = vectorstore.as_retriever(search_kwargs={"k": 12})
qa_chain_k12 = RetrievalQA.from_chain_type(
    llm = local_llm,
    chain_type= "stuff" ,
    retriever = retriever_k12,
    return_source_documents = False,
    chain_type_kwargs = {"prompt": prompt}
)

In [8]:
print("Chain input keys:", qa_chain_k5.input_keys)

Chain input keys: ['query']


# Generation Testing

In [10]:
for question, actual_pages in question_page_pairs:
    result = qa_chain_k5({'query': question})
    answer = result["result"]
    docs = result["source_documents"]
    retrieved_pages = sorted({doc.metadata["page_number"] for doc in docs})

    print("Question: ", question)
    print("Retrieved pages: ", retrieved_pages)
    print("Actual pages: ", actual_pages)
    print("LLM response: ", answer)
    print("-" * 80)


Question:  How can I add enumerations using TCL commands?
Retrieved pages:  [214, 215, 245, 308, 309]
Actual pages:  [214]
LLM response:  You are a Vivado expert. Use the following excerpts (each labeled with its page) to answer the question from a new user below.
When you quote a fact, provide the page number as well.

Excerpts:
The following table contains descriptions of all Tcl commands used to interact with links.



The following table contains descriptions of all Tcl commands used to interact with linkgroups.

*Figure 133:* **Edit Enumeration Dialog**

Select the name-value pair and use the "+" and “-” buttons on the left to add or delete
enumerations. You can change the name, radix, and values fields in the table.

The add_hw_probe_enum command, associates an enumeration name-value pair to a debug
probe. You can add add_hw_probe commands to a Tcl file, to have the definitions appear in a
separate file. The enumeration names maintain the case they were entered in, but lookup is 